In [2]:
import base64
import anthropic
from dotenv import load_dotenv
load_dotenv()
client = anthropic.Anthropic()

In [3]:
def read_file_and_encode(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_base64 = base64.standard_b64encode(pdf_file.read()).decode("utf-8")
    
    return pdf_base64

In [ ]:
def sections_1_2_instructions():
    return """
    ## Core Requirements
        - Use ONLY information from the provided V&V document - no external knowledge or invented details
        - Output a coherent, text-only narrative report with clear section headings
        - If information is missing, omit that section or note it as unavailable
        - Focus on stability-related content: look for keywords like "stability testing," "aging," "shelf life," "long-term performance"
    
    ### 1. Summary (Executive Summary)
        - Brief overview of study purpose and key findings
        - Critical data points from stability testing
        - Overall conclusion on product stability status and recommended shelf life

    ### 2. Introduction
        - Purpose and rationale of the stability study
        - Product description and context from V&V
        - Scope: duration, conditions, aspects evaluated, sample sizes
        - Regulatory/standard framework if mentioned
        - Key assumptions about test conditions
        

"""

In [ ]:
def get_system_instructions():
    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Output a coherent, text-only narrative report with clear section headings
    - If information is missing, omit that section or note it as unavailable
    - Focus on stability-related content: look for keywords like "stability testing," "aging," "shelf life," "long-term performance"

    ## Report Structure
    Generate the following sections in order:

    

    ### 3. Stability Study Methodology
    - Study design overview: what was tested, how long, under what conditions
    - Materials and samples: batch numbers, sample sizes, packaging
    - Test parameters and measurement methods
    - Acceptance criteria and pass/fail standards
    - Any deviations or protocol changes noted

    ### 4. Results
    - Present data chronologically or by parameter as organized in V&V
    - Include all relevant numerical data and observations in narrative form
    - Note trends and changes over time
    - Report compliance with acceptance criteria
    - Include visual/physical observations
    - State facts only - no interpretation here

    ### 5. Discussion
    - Analyze overall trends and their significance
    - Compare results to acceptance criteria
    - Explain implications for product quality and real-world use
    - Acknowledge study limitations
    - Provide recommendations based on findings
    - Synthesize rather than repeat raw data

    ### 6. Conclusion and Recommendations
    - Clear statement on overall product stability
    - Specific shelf life and storage condition recommendations
    - Summary of compliance with all criteria
    - Key recommendations for labeling, handling, further testing, or process changes
    - Confident closing statement

    ## Writing Guidelines
    - Write in complete sentences with clear, professional language
    - Organize content logically with smooth transitions between sections
    - Include specific data points with units when available
    - Use comparative language for trends (e.g., "remained constant," "showed slight decline")
    - Maintain objectivity and base all statements on V&V evidence
    - Ensure the report reads as a standalone document

    Generate a thorough stability report that comprehensively converts the V&V data into the standard stability report format.
    """

In [18]:


def get_anthropic_response(pdf_base64, query):
    client = anthropic.Anthropic()

    with client.messages.stream(
        model="claude-opus-4-20250514",
        max_tokens=16000,
        system=get_system_instructions(),
        thinking={
            "type": "enabled",
            "budget_tokens": 10000
        },
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "document",
                        "source": {
                            "type": "base64",
                            "media_type": "application/pdf",
                            "data": pdf_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": query
                    }
                ]
            }
        ],
    ) as stream:
        content = []
        for event in stream:
            if hasattr(event, 'delta') and hasattr(event.delta, 'text'):
                content.append(event.delta.text)

        return ''.join(content)




In [19]:
pdf_data = read_file_and_encode('docs/PRJ-0695-Silicon Change on NIP valve-Design Verification & Validation Report.pdf')
response=get_anthropic_response(pdf_data, "Generate a product stability report from the V&V document.")

In [20]:
response.split('\n\n')

['# Product Stability Report: Silicon Change on NIP Valve',
 '## 1. Summary',
 'This stability study evaluated the long-term performance of the NIP Stand-Alone Luer Activated Valve (LAV) with a new silicone septum material (SILPURAN® 6000/50 A+B, PN 590422) replacing the discontinued TSL097 Silicone QP1 50 LSR. The study demonstrated that products maintain full functionality and meet all performance specifications after accelerated aging equivalent to 5 years, including exposure to gamma irradiation up to 42.8 kGy and two EtO sterilization cycles. Based on comprehensive testing results, the new silicone septum valve is approved for a 5-year shelf life.',
 '## 2. Introduction',
 'The stability study was initiated following notification from Dow Corning regarding discontinuation of the TSL097 Silicone QP1 50 LSR raw material. Haemopharm (Bonini), the septum manufacturer, selected SILPURAN® 6000/50 A+B from Wacker as the replacement material. This study aimed to verify that the new septum